In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

/home/joaquin/Documents/GitHub/skforecast


In [ ]:
#!pip install skforecast==0.15.0

  Using cached skforecast-0.15.0-py3-none-any.whl.metadata (18 kB)
Using cached skforecast-0.15.0-py3-none-any.whl (818 kB)
  Attempting uninstall: skforecast
    Found existing installation: skforecast 0.14.0
    Uninstalling skforecast-0.14.0:
      Successfully uninstalled skforecast-0.14.0


In [1]:
# Libraries
# ==============================================================================
import timeit
import platform
import joblib
import os
import sys
import warnings
import hashlib
import inspect
import psutil
import sklearn
import numpy as np
import pandas as pd
import lightgbm
import skforecast
import plotly.express as px
import plotly.graph_objects as go
if skforecast.__version__ >= '0.14.0':
    from skforecast.recursive import ForecasterRecursive, ForecasterRecursiveMultiSeries
    from skforecast.direct import ForecasterDirect, ForecasterDirectMultiVariate
else:
    from skforecast.ForecasterAutoreg import ForecasterAutoreg
    from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries
    from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
    from skforecast.ForecasterAutoregMultiVariate import ForecasterAutoregMultiVariate
from skforecast.utils import *
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMRegressor
print('Skforecast version: ', skforecast.__version__)

Skforecast version:  0.15.0


In [2]:
# Becnchmarking functions
# ==============================================================================
class BenchmarkRunner:
    def __init__(self, output_dir="benchmarks", repeat=10):
        self.output_dir = output_dir
        self.repeat = repeat
        os.makedirs(self.output_dir, exist_ok=True)

    def get_system_info(self):
        return {
            'datetime': pd.Timestamp.now(),
            'python_version': platform.python_version(),
            'skforecast_version': skforecast.__version__,
            'numpy_version': np.__version__,
            'pandas_version': pd.__version__,
            'sklearn_version': sklearn.__version__,
            'lightgbm_version': lightgbm.__version__,
            'platform': platform.platform(),
            'processor': platform.processor(),
            'cpu_count': psutil.cpu_count(logical=True),
            'memory_gb': round(psutil.virtual_memory().total / 1e9, 2),
        }

    def hash_function_code(self, func):
        src = inspect.getsource(func)
        return hashlib.md5(src.encode()).hexdigest()

    def time_function(self, func, *args, **kwargs):
        try:
            times = timeit.repeat(
                stmt=lambda: func(*args, **kwargs),
                repeat=self.repeat,
                number=1
            )
            return {
                'avg_time': np.mean(times),
                'std_dev': np.std(times)
            }
        except Exception as e:
            warnings.warn(f"The function {func.__name__} raised an exception: {e}")
            return {
                'avg_time': float('nan'),
                'std_dev': float('nan')
            }

    def benchmark(self, func, forecaster=None, *args, **kwargs):
        forecaster_name = type(forecaster).__name__ if forecaster else np.nan
        func_name = func.__name__
        print(f"Benchmarking function: {func_name}")
        hash_code = self.hash_function_code(func)
        result_file = os.path.join(self.output_dir, f"{func_name}_benchmark.joblib")

        system_info = self.get_system_info()
        timing = self.time_function(func, *args, **kwargs)

        entry = {
            'forecaster_name': forecaster_name,
            'function_name': func_name,
            'function_hash': hash_code,
            'run_time_avg': timing['avg_time'],
            'run_time_std_dev': timing['std_dev'],
            **system_info
        }

        df_new = pd.DataFrame([entry])

        if os.path.exists(result_file):
            df_existing = joblib.load(result_file)

            cols_to_ignore = ['run_time_avg', 'run_time_std_dev', 'datetime']
            mask = (
                df_existing.drop(columns = cols_to_ignore)
                .eq(df_new.drop(columns = cols_to_ignore).loc[0, :])
                .all(axis=1)
            )

            if mask.any():
                warnings.warn("This benchmark already exists with the same hash and system info. Skipping save.")
                return df_existing

            df_combined = pd.concat([df_existing, df_new], ignore_index=True)
            joblib.dump(df_combined, result_file)
            return df_combined
        else:
            joblib.dump(df_new, result_file)
            return df_new
        

def plot_benchmark_results(df, function_name):
    """
    Plot benchmark results for a given function.
    """
    sorted_versions = sorted(df['skforecast_version'].unique(), key=lambda x: list(map(int, x.split('.'))))
    df['skforecast_version'] = pd.Categorical(df['skforecast_version'], categories=sorted_versions, ordered=True)
    df = df.sort_values("skforecast_version")
    medians = df.groupby('skforecast_version', observed=True)['run_time_avg'].median().reset_index()

    fig = px.strip(
        df,
        x="skforecast_version",
        y="run_time_avg",
        color="skforecast_version",
        hover_data=df.columns,
        stripmode='overlay',
    )
    fig.update_traces(jitter=True, marker=dict(size=10, opacity=0.7))

    fig.add_trace(
        go.Scatter(
            x=medians['skforecast_version'],
            y=medians['run_time_avg'],
            mode='lines+markers',
            line=dict(color='black', width=2),
            marker=dict(size=8),
            name='Median',
            showlegend=True
        )
    )

    fig.update_layout(
        title=f"Execution time of {function_name}",
        xaxis_title="skforecast version",
        yaxis_title="Execution time (seconds)",
        showlegend=True
    )
    fig.show()


In [3]:
print(f"Python version: {platform.python_version()}")
print(f"scikit-learn version: {sklearn.__version__}")
print(f"skforecast version: {skforecast.__version__}")
print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")
print(f"psutil version: {psutil.__version__}")
print("")

# Computer information
# ==============================================================================
print(f"Computer network name: {platform.node()}")
print(f"Machine type: {platform.machine()}")
print(f"Processor type: {platform.processor()}")
print(f"Platform type: {platform.platform()}")
print(f"Operating system: {platform.system()}")
print(f"Operating system release: {platform.release()}")
print(f"Operating system version: {platform.version()}")
print(f"Number of physical cores: {psutil.cpu_count(logical=False)}")
print(f"Number of logical cores: {psutil.cpu_count(logical=True)}")

Python version: 3.12.9
scikit-learn version: 1.4.2
skforecast version: 0.15.0
pandas version: 2.2.3
numpy version: 1.26.4
psutil version: 5.9.0

Computer network name: joaquin-HP-ProBook-440-G6
Machine type: x86_64
Processor type: x86_64
Platform type: Linux-6.11.0-21-generic-x86_64-with-glibc2.39
Operating system: Linux
Operating system release: 6.11.0-21-generic
Operating system version: #21~24.04.1-Ubuntu SMP PREEMPT_DYNAMIC Mon Feb 24 16:52:15 UTC 2
Number of physical cores: 4
Number of logical cores: 8


In [4]:
# Mock data for benchmarking
# ==========================================================
n_series = 600
len_series = 2000
series_dict = {}
rng = np.random.default_rng(321)
for i in range(n_series):
    series_dict[f'series_{i}'] = pd.Series(
        data = rng.normal(loc=20, scale=5, size=len_series),
        index=pd.date_range(
            start='2010-01-01',
            periods=len_series,
            freq='h'
        ),
        name=f'series_{i}'
    )

exog_dict = {}
rng = np.random.default_rng(321)
for k in series_dict.keys():
    exog = pd.DataFrame(
            index=series_dict[k].index
            )
    exog['day_of_week'] = exog.index.dayofweek
    exog['week_of_year'] = exog.index.isocalendar().week.astype(int)
    exog['month'] = exog.index.month
    exog_dict[k] = exog


exog_dict_prediction = {}
for k in series_dict.keys():
    exog = pd.DataFrame(
            index=pd.date_range(
                start=series_dict[k].index.max() + pd.Timedelta(hours=1),
                periods=100,
                freq='h'
            )
            )
    exog['day_of_week'] = exog.index.dayofweek
    exog['week_of_year'] = exog.index.isocalendar().week.astype(int)
    exog['month'] = exog.index.month
    exog_dict_prediction[k] = exog   


print(f"Range of dates: "
    f"{np.min([series_dict[k].index.min() for k in series_dict.keys()])} - "
    f"{np.max([series_dict[k].index.max() for k in series_dict.keys()])}"
)

print(f"Range of dates for prediction: "
      f"{np.min([exog_dict_prediction[k].index.min() for k in exog_dict_prediction.keys()])} - "
    f"{np.max([exog_dict_prediction[k].index.max() for k in exog_dict_prediction.keys()])}"
)

Range of dates: 2010-01-01 00:00:00 - 2010-03-25 07:00:00
Range of dates for prediction: 2010-03-25 08:00:00 - 2010-03-29 11:00:00


In [5]:
# Benchmarking speed ForecasterRecursiveMultiSeries methods
# ==============================================================================
if skforecast.__version__ >= '0.14.0':
    forecaster = ForecasterRecursiveMultiSeries(
        regressor=LGBMRegressor(random_state=8520, verbose=-1),
        lags=50,
        transformer_series=StandardScaler(),
        transformer_exog=StandardScaler(),
        encoding="ordinal"
    )
else:
     forecaster = ForecasterAutoregMultiSeries(
        regressor=LGBMRegressor(random_state=8520, verbose=-1),
        lags=50,
        transformer_series=StandardScaler(),
        transformer_exog=StandardScaler(),
        encoding="ordinal"
    )

def ForecasterRecursiveMultiSeries_fit():
    forecaster.fit(series=series_dict, exog=exog_dict)

def ForecasterRecursiveMultiSeries__create_train_X_y():
    forecaster._create_train_X_y(series=series_dict, exog=exog_dict)

def ForecasterRecursiveMultiSeries__create_train_X_y_single_series():
    _ = forecaster._create_train_X_y_single_series(
            y = series_dict['series_0'],
            exog = exog_dict['series_0'],
            ignore_exog = False,
        )

def ForecasterRecursiveMultiSeries_predict():
    forecaster.predict(steps=100, exog=exog_dict_prediction, suppress_warnings=True)

def ForecasterRecursiveMultiSeries__create_predict_inputs():
    _ = forecaster._create_predict_inputs(
            steps         = 100,
            exog         = exog_dict_prediction,
            check_inputs = True
        )

def ForecasterRecursiveMultiSeries__check_predict_inputs():
    check_predict_input(
        forecaster_name  = type(forecaster).__name__,
        steps            = 100,
        is_fitted        = forecaster.is_fitted,
        exog_in_         = forecaster.exog_in_,
        index_type_      = forecaster.index_type_,
        index_freq_      = forecaster.index_freq_,
        window_size      = forecaster.window_size,
        last_window      = pd.DataFrame(forecaster.last_window_),
        exog             = exog_dict_prediction,
        exog_type_in_    = forecaster.exog_type_in_,
        exog_names_in_   = forecaster.exog_names_in_,
        interval         = None,
        levels           = forecaster.series_names_in_,
        series_names_in_ = forecaster.series_names_in_,
        encoding         = forecaster.encoding
    )

In [6]:
runner = BenchmarkRunner(repeat=5, output_dir="benchmarks")
_ = runner.benchmark(ForecasterRecursiveMultiSeries_fit, forecaster= forecaster)
_ = runner.benchmark(ForecasterRecursiveMultiSeries__create_train_X_y, forecaster= forecaster)
forecaster.fit(series=series_dict, exog=exog_dict)
_ = runner.benchmark(ForecasterRecursiveMultiSeries__create_train_X_y_single_series, forecaster= forecaster)
_ = runner.benchmark(ForecasterRecursiveMultiSeries_predict, forecaster= forecaster)
_ = runner.benchmark(ForecasterRecursiveMultiSeries__create_predict_inputs, forecaster= forecaster)
_ = runner.benchmark(ForecasterRecursiveMultiSeries__check_predict_inputs, forecaster= forecaster)

Benchmarking function: ForecasterRecursiveMultiSeries_fit
Benchmarking function: ForecasterRecursiveMultiSeries__create_train_X_y
Benchmarking function: ForecasterRecursiveMultiSeries__create_train_X_y_single_series
Benchmarking function: ForecasterRecursiveMultiSeries_predict
Benchmarking function: ForecasterRecursiveMultiSeries__create_predict_inputs
Benchmarking function: ForecasterRecursiveMultiSeries__check_predict_inputs


In [7]:
results_fit = joblib.load("benchmarks/ForecasterRecursiveMultiSeries_fit_benchmark.joblib")
results_create_train_X_y = joblib.load("benchmarks/ForecasterRecursiveMultiSeries__create_train_X_y_benchmark.joblib")
results_create_train_X_y_single_series = joblib.load("benchmarks/ForecasterRecursiveMultiSeries__create_train_X_y_single_series_benchmark.joblib")
results_predict = joblib.load("benchmarks/ForecasterRecursiveMultiSeries_predict_benchmark.joblib")

In [8]:
# Plot results
# ==============================================================================
all_results = [
    results_fit,
    results_create_train_X_y,
    results_create_train_X_y_single_series,
    results_predict
]

for df in all_results:
    function_name = df['function_name'].values[0]
    plot_benchmark_results(df, function_name)